<a href="https://colab.research.google.com/github/EnzoItaliano/calculoNumericoEmPython/blob/master/Lista_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade Tecnológica Federal do Paraná  
Professor: Wellington José Corrêa  
Orientando: Enzo Dornelles Italiano  
Cálculo Numérico

Inicialmente precisamos executar uma vez os códigos abaixo

#Códigos

In [0]:
!pip3 install prettymatrix
!pip install mpld3
!pip install "git+https://github.com/javadba/mpld3@display_fix"
import copy
import math
import mpld3
import numpy as np
from sympy import *
import prettymatrix
from mpld3 import plugins
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from numpy.polynomial import Polynomial as P
x = symbols('x')
def Lagrange(pontos, valor, f):
    Pn = 0
    print("Polinômios coeficientes")
    for i in range(len(pontos)):
        mult = 1
        multp = 1
        div = 1
        for j in range(len(pontos)):
            if i == j: continue
            mult *= P([-pontos[j][0], 1])
            multp *= x - pontos[j][0]
            div *= pontos[i][0] - pontos[j][0]
        print("\n>>>>>>>L[%a]<<<<<<<" % i)
        pprint(multp/div)
        Pn = Pn + pontos[i][1] * (mult // div)
    print("Polinômio interpolador de Lagrange p(x) = ", end="")
    poli = list(Pn)
    for i in range(len(poli)):
        print(abs(round(poli[i],8)),end="")
        if i == 0: print(" ",end="")
        elif i == 1: print("x ", end="")
        else: print("x**%o"%i, end=" ")
        if i != len(poli)-1:
            if poli[i+1] >= 0:
                print("+ ", end="")
            else:
                print("- ", end="")
    print("\n")
    print("Polinômio interpolador avaliado em x =",valor,", é P("+str(valor)+") =" ,Pn(valor))

    if f != 0:
        f = diff(f,x,len(poli))
        # print(simplify(f))
        maior = abs(f.subs(x,pontos[0][0]))
        if abs(f.subs(x,pontos[len(pontos)-1][0])) > maior:
            maior = abs(f.subs(x,pontos[len(pontos)-1][0]))
        mult = 1
        for i in range(len(pontos)):
            mult *= abs(valor-pontos[i][0])
        E = mult * maior / factorial(len(poli))
        print("\nLimitante")
        print("|E("+str(valor)+")| <= ",E.evalf())

def plotL(pontos, xi, xf):
    l = []
    for i in range(len(pontos)):
        multp = 1
        div = 1
        for j in range(len(pontos)):
            if i == j: continue
            multp *= x - pontos[j][0]
            div *= pontos[i][0] - pontos[j][0]
        l.append(multp/div)
    return l

def graficoLagrange(pontos):
    Pn = 0
    for i in range(len(pontos)):
        mult = 1
        div = 1
        for j in range(len(pontos)):
            if i == j: continue
            mult *= P([-pontos[j][0], 1])
            div *= pontos[i][0] - pontos[j][0]
        Pn = Pn + pontos[i][1] * (mult // div)
    return Pn

def Newton(pontos, valor, f):
    dif = []
    for i in range(len(pontos)):
        dif.append([])
    for i in range(len(pontos)):
        dif[0].append(pontos[i][1])
    for i in range(len(pontos)-1):
        for j in range(len(pontos)-(i+1)):
            dif[i+1].append((dif[i][j+1]-dif[i][j])/(pontos[j+i+1][0]-pontos[j][0]))
    
    Table = PrettyTable()
    points=[]
    for i in range(len(pontos)):
        points.append(pontos[i][0])
    Table.add_column("xk", points)
    for k in range(len(dif)):
        while len(dif[k]) < len(pontos):
            dif[k].append("-")
        Table.add_column("Dif_"+str(k),dif[k])

    print("Tabela")
    print(Table)

    Pn = dif[0][0]
    for i in range(1,len(dif)):
        temp = 1
        for j in range(i):
            temp *= (x-pontos[j][0])
        temp *= dif[i][0]
        Pn += temp
    
    print("Polinômio interpolador p(x) = ",end="")
    print(simplify(Pn))

    print("Polinômio interpolador avaliado em x = "+str(valor)+" é p("+str(valor)+") = ", end="")
    print(round(Pn.subs(x,valor),8))

    if f != 0:
        f = diff(f,x,degree(Pn,x)+1)
        # print(simplify(f))
        maior = abs(f.subs(x,pontos[0][0]))
        if abs(f.subs(x,pontos[len(pontos)-1][0])) > maior:
            maior = abs(f.subs(x,pontos[len(pontos)-1][0]))
        mult = 1
        for i in range(len(pontos)):
            mult *= abs(valor-pontos[i][0])
        E = mult * maior / factorial(degree(Pn,x)+1)
        print("\nLimitante")
        print("|E("+str(valor)+")| <= ",E.evalf())

def graficoNewton(pontos):
    dif = []
    for i in range(len(pontos)):
        dif.append([])
    for i in range(len(pontos)):
        dif[0].append(pontos[i][1])
    for i in range(len(pontos)-1):
        for j in range(len(pontos)-(i+1)):
            dif[i+1].append((dif[i][j+1]-dif[i][j])/(pontos[j+i+1][0]-pontos[j][0]))
    
    Pn = dif[0][0]
    for i in range(1,len(dif)):
        temp = 1
        for j in range(i):
            temp *= (x-pontos[j][0])
        temp *= dif[i][0]
        Pn += temp
    return Pn

def NewtonGregory(pontos, valor, f):
    intervalo = pontos[1][0] - pontos[0][0]
    for i in range(1,len(pontos)):
        if pontos[i][0] - pontos[i-1][0] != intervalo:
            return print("Valores de X não são equidistantes")
    
    dif = []
    for i in range(len(pontos)):
        dif.append([])
    for i in range(len(pontos)):
        dif[0].append(pontos[i][1])
    for i in range(len(pontos)-1):
        for j in range(len(pontos)-(i+1)):
            dif[i+1].append((dif[i][j+1]-dif[i][j]))

    Table = PrettyTable()
    points=[]
    for i in range(len(pontos)):
        points.append(pontos[i][0])
    Table.add_column("xk", points)
    for k in range(len(dif)):
        while len(dif[k]) < len(pontos):
            dif[k].append("-")
        Table.add_column("Dif_"+str(k),dif[k])

    print("Tabela")
    print(Table)
    
    Pn = dif[0][0]
    for i in range(1,len(dif)):
        temp = 1
        for j in range(i):
            temp *= (x-pontos[j][0])
        temp *= (dif[i][0]/(factorial(i)*intervalo**i))
        Pn += temp
    
    print("Polinômio interpolador p(x) = ",end="")
    print(Pn)

    print("Polinômio interpolador avaliado em x = "+str(valor)+" é p("+str(valor)+") = ", end="")
    print(round(Pn.subs(x,valor),8))

    if f != 0:
        f = diff(f,x,degree(Pn,x)+1)
        # print(simplify(f))
        maior = abs(f.subs(x,pontos[0][0]))
        if abs(f.subs(x,pontos[len(pontos)-1][0])) > maior:
            maior = abs(f.subs(x,pontos[len(pontos)-1][0]))
        mult = 1
        for i in range(len(pontos)):
            mult *= abs(valor-pontos[i][0])
        E = mult * maior / factorial(degree(Pn,x)+1)
        print("\nLimitante")
        print("|E("+str(valor)+")| <= ",E.evalf())

def graficoNG(pontos):
    intervalo = pontos[1][0] - pontos[0][0]
    for i in range(1,len(pontos)):
        if pontos[i][0] - pontos[i-1][0] != intervalo:
            return print("Valores de X não são equidistantes")
    
    dif = []
    for i in range(len(pontos)):
        dif.append([])
    for i in range(len(pontos)):
        dif[0].append(pontos[i][1])
    for i in range(len(pontos)-1):
        for j in range(len(pontos)-(i+1)):
            dif[i+1].append((dif[i][j+1]-dif[i][j]))
    
    Pn = dif[0][0]
    for i in range(1,len(dif)):
        temp = 1
        for j in range(i):
            temp *= (x-pontos[j][0])
        temp *= (dif[i][0]/(factorial(i)*intervalo**i))
        Pn += temp
    return Pn

def sistLinear(G, B, ordem):
    y = symbols('y:'+str(ordem))
    mY = []
    for i in range(len(y)):
        mY.append(y[i])
    D = np.linalg.det(G)
    tempG = G.copy()
    for j in range(ordem):
        for i in range(ordem):
            tempG[i][j] = B[i]
        tempD = np.linalg.det(tempG)
        tempG = G.copy()
        mY[j] = round(tempD/D, 8)
    mTemp = []
    for i in range(len(mY)):
        mTemp.append([mY[i]])
    mY = mTemp.copy()
    mY = np.asarray(mY)
    return mY

def spline(pontos, valor):
    h = []
    for i in range(1,len(pontos)):
        h.append(pontos[i][0] - pontos[i-1][0])
    M = np.zeros((len(h)-1,len(h)-1))
    for i in range(len(h)-1):
        if i == 0:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i+1] = h[i+1]
        elif i == len(h)-2:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i-1] = h[i]
        else:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i-1] = h[i]
            M[i][i+1] = h[i+1]

    print(prettymatrix.matrix_to_string(M, name='Matriz = '))
    B = np.zeros((len(h)-1,1))
    for i in range(1,len(h)):
        B[i-1][0] = 6*((pontos[i+1][1]-pontos[i][1])/h[i]) - 6*((pontos[i][1]-pontos[i-1][1])/h[i-1])

    print(prettymatrix.matrix_to_string(B, name='B = '))
    mu = sistLinear(M, B, len(h)-1)
    print("Spline natural: \u03BC0 = 0, \u03BC"+str(len(h))+" = 0\n")
    print("Resolvendo o sistema linear M*Y=B, temos:")
    print('\u03BC1 = ', mu[0][0])
    print('\u03BC2 = ', mu[1][0])

    alpha = np.zeros(len(h))
    beta  = np.zeros(len(h))
    gamma = np.zeros(len(h))
    for i in range(len(h)):
        if i == 0:
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((mu[i][0]/6)*h[i]) - ((0/3)*h[i])
            beta[i] = 0/2
            gamma[i] = (mu[i][0]-0)/(6*h[i])
        elif i == len(mu):
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((0/6)*h[i]) - ((mu[i-1]/3)*h[i])
            beta[i] = mu[i-1][0]/2
            gamma[i] = (0-mu[i-1][0])/(6*h[i])
        else:
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((mu[i][0]/6)*h[i]) - ((mu[i-1]/3)*h[i])
            beta[i] = mu[i-1][0]/2
            gamma[i] = (mu[i][0]-mu[i-1][0])/(6*h[i])
    
    i = np.linspace(0,len(alpha)-1,len(alpha))
    Table = PrettyTable()
    Table.add_column("i",i)
    Table.add_column("\u03B1",alpha)
    Table.add_column("\u03B2",beta)
    Table.add_column("\u03B3",gamma)
    print("\nCoeficientes dos polinomios da spline:")
    print(Table)

    S = []
    for i in range(len(alpha)):
        S.append(pontos[i][1] + (alpha[i]*(x-pontos[i][0])) + (beta[i]*(x-pontos[i][0])**2) + (gamma[i]*(x-pontos[i][0])**3))
    
    print("\nSpline cúbica natural:\n")
    for i in range(len(S)):
        print("P"+str(i)+"(x) = "+str(simplify(S[i]))+" , Intervalo=["+str(pontos[i][0])+","+str(pontos[i+1][0])+"]")
    print("")

    c = 0
    for i in range(1,len(pontos)):
        intervalo = [pontos[i-1][0],pontos[i][0]]
        if valor >= intervalo[0] and valor < intervalo[1]:
            c = copy.copy(i)
            break
    print("Queremos encontrar o valor para f("+str(valor)+") então devemos usar P"+str(c-1)+" pois x = "+str(valor)+" está contido no intervalo = ",intervalo)
    print("\nLogo, a função em x = "+str(valor)+" é aproximadamente: ",S[1].subs(x,valor))

def graficoSpline(pontos, valor):
    h = []
    for i in range(1,len(pontos)):
        h.append(pontos[i][0] - pontos[i-1][0])
    # print(len(h))
    M = np.zeros((len(h)-1,len(h)-1))
    for i in range(len(h)-1):
        if i == 0:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i+1] = h[i+1]
        elif i == len(h)-2:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i-1] = h[i]
        else:
            M[i][i]   = 2*(h[i]+h[i+1])
            M[i][i-1] = h[i]
            M[i][i+1] = h[i+1]

    B = np.zeros((len(h)-1,1))
    for i in range(1,len(h)):
        B[i-1][0] = 6*((pontos[i+1][1]-pontos[i][1])/h[i]) - 6*((pontos[i][1]-pontos[i-1][1])/h[i-1])

    mu = sistLinear(M, B, len(h)-1)

    alpha = np.zeros(len(h))
    beta  = np.zeros(len(h))
    gamma = np.zeros(len(h))
    for i in range(len(h)):
        if i == 0:
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((mu[i][0]/6)*h[i]) - ((0/3)*h[i])
            beta[i] = 0/2
            gamma[i] = (mu[i][0]-0)/(6*h[i])
        elif i == len(mu):
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((0/6)*h[i]) - ((mu[i-1]/3)*h[i])
            beta[i] = mu[i-1][0]/2
            gamma[i] = (0-mu[i-1][0])/(6*h[i])
        else:
            alpha[i] = ((pontos[i+1][1]-pontos[i][1])/h[i]) - ((mu[i][0]/6)*h[i]) - ((mu[i-1]/3)*h[i])
            beta[i] = mu[i-1][0]/2
            gamma[i] = (mu[i][0]-mu[i-1][0])/(6*h[i])

    S = []
    for i in range(len(alpha)):
        # print(pontos[i][1] +(alpha[i]*(x-pontos[i][0])))
        S.append(pontos[i][1] + (alpha[i]*(x-pontos[i][0])) + (beta[i]*(x-pontos[i][0])**2) + (gamma[i]*(x-pontos[i][0])**3))
    
    c = 0
    for i in range(1,len(pontos)):
        intervalo = [pontos[i-1][0],pontos[i][0]]
        if valor >= intervalo[0] and valor < intervalo[1]:
            c = copy.copy(i)
            break

    Pn = S
    return Pn,c

#Interpolação

## 1. Polinônimo de Lagrange

O procedimento aqui é Lagrange(pontos,valor,f(x))

Onde pontos é a tabela descrita na forma de matriz, valor é o ponto a ser avaliado e $f(x)$ é a função na qual é possível estimar 
o erro. Quando se deseja apenas obter o polinômio interpolador de Lagrange, façamos $f(x)=0$.

Consideremos dois casos:

(a) Quando a função $f(x)$ é desconhecida. Neste caso, tomamos f=0 no algoritmo Lagrange(pontos,valor,f(x)).

Exemplo: Conhecendo-se a seguinte tabela

| x    | -1 | 0 | 3 |
|------|----|---|---|
| f(x) | 15 | 8 | 1 |

Determine o polinômio interpolador na forma de Lagrange e obtenha uma aproximação para $f(1)$.

Solução: Como a função $f$ é desconhecida, segundo a instrução acima, consideremos $f=0$ e valor = 1:

In [0]:
def f(x): return 0
valor = 1

Em seguida, consideremos a tabela dada na forma de matriz:

In [0]:
pontos = [[-1,15],[0,8],[3,-1]]

Logo, basta usar o comando Lagrange(dados,valor,f(x)):

In [0]:
Lagrange(pontos,valor,f(x))

Sabemos que os polinômios coeficientes tem a propriedade que $L_i(x_i)=1$ e $L_i(x_j)=0$ para
$i\neq j$. Podemos ver isso graficamente pelo comando
plotL(pontos, xi, xf), onde $x_i$ é o x inicial do gráfico e $x_f$ é o final.

In [0]:
xi = -1.5
xf = 3.5
result = plotL(pontos, xi, xf)

fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
    
y = np.zeros((len(result),len(z)))
for i in range(len(result)):
    for j in range(len(z)):
        y[i][j] = (result[i].subs(x,z[j]))
    ax.plot(z,y[i], label=str(result[i]))

ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

Para plotar o gráfico do polinômio de Lagrange, basta usar o seguinte comando:

In [0]:
result = graficoLagrange(pontos)
xi = -4
xf = 4
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result(z[i]))

a = []
w = []
for i in range(len(pontos)):
    a.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(a,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(valor,result(valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

(b) Caso em que a $f(x)$ é apresentada. Neste caso, é possível avaliar o erro cometido na interpolação.

Exemplo: Considere a função $f(x)=\frac{3+x}{1+x}$ definida nos pontos conforme a tabela:

In [0]:
pontos = [[0.1,2.82],[0.2,2.67],[0.4,2.43]]

Determine o polinomio interpolador de $f(x)$, usando a fórmula de Lagrange. Em seguida, avalie $f(0.25)$ e um limitante superior para o erro.

Solução: Definamos a função $f$ e o valor = 0.25:

In [0]:
def f(x): return (3+x)/(1+x)
valor = 0.25

Logo, basta usar o comando Lagrange(pontos,valor,f(x)):

In [0]:
Lagrange(pontos, valor, f(x))

Sabemos que os polinômios coeficientes tem a propriedade que $L_i(x_i)=1$ e $L_i(x_j)=0$ para $i\neq j$. Podemos ver isso graficamente pelo comando plotL(pontos)

In [0]:
xi = -0.4
xf = 0.9
result = plotL(pontos, xi, xf)

fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
    
y = np.zeros((len(result),len(z)))
for i in range(len(result)):
    for j in range(len(z)):
        y[i][j] = (result[i].subs(x,z[j]))
    ax.plot(z,y[i], label=str(result[i]))

ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

Para plotar o gráfico do polinômio de Lagrange, basta usar o seguinte comando:

In [0]:
result = graficoLagrange(pontos)
xi = -1
xf = 1.5
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result(z[i]))

a = []
w = []
for i in range(len(pontos)):
    a.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(a,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(valor,result(valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

Como neste exemplo, $f(x)$ é dada, façamos os gráfico de $f(x)$ e $p(x)$ empregando o comando:

In [0]:
result = graficoLagrange(pontos)
xi = -0.5
xf = 1.5
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result(z[i]))

a = []
for i in range(len(z)):
    a.append(f(x).subs(x,z[i]))

b = []
w = []
for i in range(len(pontos)):
    b.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(b,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(z,a, label="Função f(x)")
ax.plot(valor,result(valor), "g*", markersize=6, label="Estimativa")
ax.plot(valor,f(x).subs(x,valor), "yo", label="Valor exato")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

## 2. Interpolação: Diferenças Divididas: Fórmula de Newton

O procedimento aqui é Newton(pontos,valor,f(x))

Determine o polinômio interpolador usando a fórmula de Newton. Além disso, avalie $f(0.7)$ onde $f(x)=e^x+sin(x)$ e exiba um limitante superior para o erro. Caso apenas deseje encontrar o polinômio interpolador, considere $f(x)=0$.

In [0]:
pontos = [[0,1],[.5,2.12],[1,3.55]]

Solução: Inicialmente, definamos $f(x)$ e valor:

In [0]:
def f(x): return exp(x) + sin(x)
valor = 0.7

Logo,

In [0]:
Newton(pontos,valor,f(x))

Para plotar o gráfico do polinômio interpolador, basta usar o seguinte comando:

In [0]:
result = graficoNewton(pontos)
xi = -1
xf = 2
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result.subs(x,z[i]))

a = []
w = []
for i in range(len(pontos)):
    a.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(a,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(valor,result.subs(x,valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

Para plotar o gráfico de $f(x)$ e $p(x)$, basta usar o comando:

In [0]:
result = graficoNewton(pontos)
xi = -1
xf = 2
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result.subs(x,z[i]))

a = []
for i in range(len(z)):
    a.append(f(x).subs(x,z[i]))

b = []
w = []
for i in range(len(pontos)):
    b.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(b,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(z,a, label="Função f(x)")
ax.plot(valor,f(x).subs(x,valor), "yo", markersize=7,label="Valor exato")
ax.plot(valor,result.subs(x,valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

## 3. Polinômio de Newton-Grégory

O procedimento aqui é NewtonGregory(pontos,valor,f(x))

Exemplo: Considere a função $f(x)=\frac{1}{1+x}$ tabelada como segue

In [0]:
pontos = [[0,1],[1,.5],[2,2/3]]

Determine o polinômio interpolador pela fórmula de Newton-Gregory, avalie $f(1,3)$ e exiba um limitante superior para o erro.

Solução: Inicialmente, definamos a função $f(x)$:

In [0]:
def f(x): return x/(1+x)
valor = 1.3

In [0]:
NewtonGregory(pontos, valor, f(x))

Para plotar o gráfico do polinômio interpolador, basta usar o seguinte comando:

In [0]:
result = graficoNG(pontos)
xi = -1
xf = 3
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result.subs(x,z[i]))

a = []
w = []
for i in range(len(pontos)):
    a.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(a,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(valor,result.subs(x,valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

Finalmente, plotamos o gráfico de $f(x)$ e $p(x)$:

In [0]:
result = graficoNG(pontos)
xi = -0.4
xf = 3
fig, ax = plt.subplots()

z = np.arange(xi,xf,0.001)
y = []
for i in range(len(z)):
    y.append(result.subs(x,z[i]))

a = []
for i in range(len(z)):
    a.append(f(x).subs(x,z[i]))

b = []
w = []
for i in range(len(pontos)):
    b.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(b,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(z,y, label='Polinômio Interpolador P(x)')
ax.plot(z,a, label="Função f(x)")
ax.plot(valor,f(x).subs(x,valor), "yo", markersize=7,label="Valor exato")
ax.plot(valor,result.subs(x,valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()

## 4. Splines

Usaremos o comando spline(pontos,valor), que nos dará além da spline, o sistema linear e todos os coeficientes necessários para a obtenção da spline.

O procedimento spline_grafico(pontos,valor) fornece a spline avaliada em x = valor e exibe o gráfico da spline no intervalo $[a,b]$.

Exemplo: Ajuste os dados da tabela abaixo com uma spline cúbica natural.

In [0]:
pontos = [[3,2.5],[4.5,1],[7,2.5],[9,.5]]

Calcule a função em x = 5.

Solução: De fato, 

In [0]:
valor = 5

In [0]:
spline(pontos,valor)

E o gráfico de p(x) é:

In [0]:
result,c = graficoSpline(pontos, valor)
xi = -1
xf = 3
fig, ax = plt.subplots()

for i in range(len(pontos)-1):
    z = np.arange(pontos[i][0],pontos[i+1][0],0.001)
    y = []
    for j in range(len(z)):
        y.append(result[i].subs(x,z[j]))
    ax.plot(z,y, label='Polinômio Interpolador P'+str(i)+'(x)')

a = []
w = []
for i in range(len(pontos)):
    a.append(pontos[i][0])
    w.append(pontos[i][1])

ax.plot(a,w, "r*", markersize=6, label="Pontos da tabela")
ax.plot(valor,result[c-1].subs(x,valor), "g*", markersize=6, label="Estimativa")
ax.legend()
ax.grid()
plugins.connect(fig, plugins.MousePosition(fontsize=14))

mpld3.display()